<a href="https://colab.research.google.com/github/PETER-NDIRITU/SAM--GEO/blob/main/Segment_Anything_Model_SAM_Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#lets import the necessary libraries
!pip install segment-geospatial

In [2]:
#lets import the necessary libraries
import os
import leafmap
!pip install segment-geospatial
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [3]:
m = leafmap.Map(center=[-1, 37], zoom=8)
m.add_basemap("SATELLITE")

if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [36.5, 0, 38.5, -2]

m

Map(center=[-1, 37], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [4]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=7, source="SATELLITE", overwrite=True)
m.layers[-1].visible = False  # we turn off the basemap
m.add_raster(image, layer_name="image")
m

Downloaded image 1/2
Downloaded image 2/2
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


Map(center=[-1, 37], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [5]:
# #lets introduce SAM Class
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

In [6]:
#lets not segment the image
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3,3), mask_multiplier=255
)


100%|██████████| 1/1 [10:24<00:00, 624.07s/it]


In [10]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [11]:
#lets save to .shp
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)


In [12]:
# lets visualize the results
style = {
    "color": "#3388ff",
    "weight" : 2,
    "fillColor" : "#7c4185",
    "fillOpacity" : 0.5,
}

m.add_vector (vector, layer_name="Vector", style=style)
m

Map(bottom=33250.0, center=[-0.9997051308419566, 37.49908447265626], controls=(ZoomControl(options=['position'…